# Web request for words similar to "Climate change"

In [2]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import json
import nltk
from nltk.stem import LancasterStemmer
import pandas as pd

We are now going to perform the query for words related to "climate change". The website from which we are quering, uses multiple algorithms to extract the words that are the most similar to "climate change". The further away they are on the list, the least similar they are.

In [3]:
# Make the request
r = requests.get('https://relatedwords.org/relatedto/climate%20change')

# Parse content
soup = BeautifulSoup(r.text, 'html.parser')

# Extract words with score and source
for words in soup.find_all('script', attrs={"id" : "preloadedDataEl", "type" : "text/json"}):
    words_json_format = json.loads(words.string)

Now that we have extracted what we wanted, we will transfrom the content into something we can use for analysis.

In [4]:
# Transform into python list fo scores associated to words
words_with_score = []
words_only = []

for term in words_json_format['terms']:
    words_with_score.append((term['word'], term['score']))
    words_only.append(term['word'])

# Stemming

In [5]:
# Stemming
def stem_list(words_list):
    stemmed_words = []
    lancaster = LancasterStemmer()

    for word in words_only:
        stemmed_words.append(lancaster.stem(word))
        
    return stemmed_words

# Extracting and stemming data from dataset

In [6]:
# Load data
df = pd.read_json('data/quotebank/quotes-2020.json.bz2', lines=True, compression='bz2', chunksize =10)

In [7]:
# Setup set of words we're using for searching
stemmed_words = stem_list(words_only)
stemmed_words = set(stemmed_words[:10]) #Only use 10 first words for now

In [15]:
#%%time
#climate_quotes = []
#for row in df:
#    print(row['quotation'])
    #df = chunk[chunk.quotation.str).contains('climate', case=False, na=False)]
    #climate_quotes.append(df)
# load data using Python JSON module
with open('data/quotebank/quotes-2020.json.bz2','r') as f:
    data = json.loads(f.read())
# Normalizing data
multiple_level_data = pd.json_normalize(data, record_path =['Results'], meta =['original_number_of_clusters','Scaler','family_min_samples_percentage'], meta_prefix='config_params_', record_prefix='dbscan_')
# Saving to CSV format
multiple_level_data.to_csv('multiplelevel_normalized_data.csv', index=False)

AttributeError: 'JsonReader' object has no attribute 'id'